# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902064


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:22,  2.85s/it]

Rendering models:  13%|█▎        | 4/31 [00:06<00:58,  2.17s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:41,  1.59s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:28,  1.15s/it]

...

Rendering models:  74%|███████▍  | 23/31 [00:09<00:01,  5.54it/s]

Rendering models:  77%|███████▋  | 24/31 [00:10<00:01,  6.16it/s]

Rendering models:  81%|████████  | 25/31 [00:10<00:01,  5.50it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  7.28it/s]

equidad1                               0.310609
kayleebug2017                          0.002285
JessieDawn                             0.004817
not-logged-in-0c20e1f083ae2ffe33bd     0.004587
Thedunkmasta                           0.003817
ElisabethB                             0.065760
not-logged-in-0997cfcba8da23453381     0.043765
not-logged-in-c66f4fc0a6fdbc2ff731     0.001314
kebnekaise                             0.001635
cedar-phoenix                          0.000577
not-logged-in-615361e970b43bb9dc1e     0.034411
sn346808                               0.026444
stellarastronomyisstellar             16.813700
kevinlacaille                          0.003155
bnamumba                               0.002304
ControlledChaos                        0.033344
connorstone628                         0.001164
fredsta126                             0.017936
HenriqueDornelles                      0.105918
BottledFarts                           0.020972
not-logged-in-8a93f230391960dff6c1     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())